In [1]:
import os
CODE_DIR = '../../sber-swap'
os.chdir(f'./{CODE_DIR}')

In [2]:
import cv2
import torch
import time
import os

from utils.inference.image_processing import crop_face, get_final_image, show_images
from utils.inference.video_processing import read_video, get_target, get_final_video_encode, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

/opt/conda/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(


In [3]:
from argparse import Namespace

In [4]:
def load_model(use_sr=False):
    app = Face_detect_crop(name='antelope', root='./insightface_func/models')
    app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

    # main model for generation
    G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
    G.eval()
    G.load_state_dict(torch.load('weights/G_unet_2blocks.pth', map_location=torch.device('cpu')))
    G = G.cuda()
    G = G.half()

    # arcface model to get face embedding
    netArc = iresnet100(fp16=False)
    netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
    netArc=netArc.cuda()
    netArc.eval()

    # model to get face landmarks
    handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

    # model to make superres of face, set use_sr=True if you want to use super resolution or use_sr=False if you don't
    model = None
    if use_sr:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        torch.backends.cudnn.benchmark = True
        opt = TestOptions()
        #opt.which_epoch ='10_7'
        model = Pix2PixModel(opt)
        model.netG.train()
    
    return app, G, netArc, handler, model

In [5]:
def run_inference(source_full, path_to_video, use_sr, aligned, OUT_VIDEO_NAME):
    # check, if we can detect face on the source image
    crop_size = 224 # don't change this
    BS = 60
    app, G, netArc, handler, model = load_model(use_sr=True)
    try:
        if not aligned:
            source = crop_face(source_full, app, crop_size)[0]
            source = [source]
        else:
            if source_full.shape[0] > 224:
                source = [cv2.resize(source_full, (224, 224))]
            else:
                source = [source_full]
        print("Everything is ok!")
    except TypeError:
        print("Bad source images")
    
    full_frames, fps = read_video(path_to_video)
    target = get_target(full_frames, app, crop_size)

        
    final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,
                                                                                       source,
                                                                                       target,
                                                                                       netArc,
                                                                                       G,
                                                                                       app,
                                                                                       set_target = False,
                                                                                       crop_size=crop_size,
                                                                                       BS=BS)
    if use_sr:
        final_frames_list = face_enhancement(final_frames_list, model)
        
    output_file_name = get_final_video_encode(final_frames_list,
                    crop_frames_list,
                    full_frames,
                    tfm_array_list,
                    OUT_VIDEO_NAME,
                    fps, 
                    handler)
#     add_audio_from_another_video(path_to_video, OUT_VIDEO_NAME, "audio")
    return output_file_name

In [6]:
import gradio as gr



In [7]:
examples = [
    ['examples/images/320_persona.png', 'examples/videos/dance.mp4', True, True, "examples/results/result.mp4" ],
    ['examples/images/iu.jpeg', 'examples/videos/idol1.mp4', True, False, "examples/results/iu.mp4" ],
    ['examples/images/iu.jpeg', 'examples/videos/idol2.mp4', True, False, "examples/results/iu2.mp4" ],
]

In [8]:
gr.Interface(
    run_inference, 
    inputs=[
        gr.inputs.Image(),
        gr.Video(),
        gr.Checkbox(),
        gr.Checkbox(),
        gr.Textbox(),
    ],
    outputs= "playable_video",
    examples=examples,).launch(server_name='0.0.0.0')
        
    

/opt/conda/lib/python3.8/site-packages/gradio/inputs.py:270: DeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://localhost:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f52306639d0>, 'http://localhost:7860/', None)